# 大语言模型原理简介

**大语言模型**（**large language model**，**LLM**）的出现使得**人工智能**（**artificial intelligence**，**AI**）的发展进入了新的纪元，特别是OpenAI的GPT系列模型经过不断迭代进化，让人们看到了基于大语言模型的人工智能的无限可能。

实际上，大语言模型的出现并非突然，而是有深厚的“小语言”模型的基础。实际上正如**GPT**（**Generative Pre-trained Transformers**）包括**BERT**（**Bidirectional Encoder Representations from Transformers**）名称中所暗示的，两者都是基于谷歌的几位科学家2017年的论文《Attention is All You Need》所提出的Transformer架构：

![Transformer架构](pic/transformer.png)

而这一架构的提出则是将之前的**嵌入**（**embedding**）这一方法（如Word2Vec）进行了拓展。

在传统的嵌入方法中，句子首先被分解为一些**标记**（**token**）：
* 对于英文而言，标记可以是一个单词，或者是比单词更细的成分（比如把shocked分开为shock和ed）；
* 对于中文而言，通常每个字是一个token，或者可以通过分词技术进行分词（但在LLM中似乎没有必要）。
然后每个标记被转化为一个向量，从而把文本转化为了一个可以被计算的对象。

然而以上方法有其弊端：有的词天然是多义的，无法考虑上下文。比如，“学长可以写出更长的文章”这个句子中，两个“长”甚至连读音都不相同，但是在之前的嵌入模型中，“长”字可能被翻译为同一个向量。当然也许可以使用分词和n-gram的方法部分解决这一问题，然而会导致模型的复杂度急剧上升。

为此，Transformer架构中创造性地使用了**注意力**（**attention**）机制解决这一问题。我们在这里简要介绍一下注意力机制的简单直觉，不再对其数学部分做更多的解读，理解这一算法背后的直觉可以帮助我们理解大语言模型以及更好地使用大语言模型。

## 注意力机制

正如字面意思，注意力机制是模仿了人脑的注意力。比如，当我们阅读一段文字：

> 在我的后园，可以看见墙外有两株树，一株是枣树，还有一株也是枣树。

当我们看到“两棵树”，自然会在后文中**查找**（**Query**）是哪两棵，我们可能会以“树”这个**关键字**（**Key**）去查找，然后找到了后文的两个“枣树”，进而在脑海中形成了“两颗枣树”这一映像，或者在计算机语言中，也就是“键”所对应的“**值**”（**Value**）。

在Attention中正是使用以上Q、K、V三个工具，通过Q和V两个矩阵相乘，经过Softmax变成一个注意力的概率，再与V相乘得到最终的注意力结果。

由于在一个句子中，根据不同的任务，我们的注意力可能集中在不同的token上面，所以在实际使用时，使用的是“多头注意力”（multi-head attention），即将多个注意力工具合在一起使用。注意力的架构如下图所示：

![Attention](pic/attention.png)

根据不同的任务，注意力可以分为两种：交叉注意力和自注意力。

* 交叉注意力（cross attention）：Q来自于一个序列，而K、V来自于另一个序列：

![Cross-Attention](pic/cross_attention.png)

* 自注意力（self-attention）：Q、K、V来自于同一个序列：

![Self-Attention](pic/self_attention.png)

## LLM的训练过程

注意力机制以及Transformer架构为大语言模型提供了技术保障，然而还有两个对LLM训练至关重要的问题没有解决：

1. 如何训练模型？需要定义好损失函数（训练目标是什么？）
2. 数据源从哪里来？

对第二个问题的回答来源于第一个问题，不同的训练任务、目标需要有不同的损失函数，从而也需要不同的数据。

在Transformer的原文中，主要解决的是翻译问题，那么只需要提供原文和译文，目标函数就是让机器根据原文翻译的译文与人翻译的尽量相近。为此，需要同时使用编码器和解码器。

而在Transformer提出后的一年，有两个工作放弃了编码器-解码器的结构，而是只使用了其中的一种：

* BERT：只使用了编码器
* GPT：只使用了解码器

具体而言，BERT由于只提供编码器作为预训练模型使用，因而只有预训练阶段；而GPT的目的是生成，因而除了预训练，还有有监督微调（supervised fine-tuning）。

在预训练阶段，BERT和GPT使用了类似的策略，其中：

* BERT完成两种任务
  1. 遮蔽语言模型（MLM），类似于完形填空，比如：
    > [CLS]一条大[MASK]波浪宽[SEP]
  2. 下句预测（NSP）：预测下个句子
* GPT主要预测下一个token（NTP）。

预训练之后，GPT还会进行进一步的有监督微调，通过文本分类、问答、自然语言推断等有监督任务继续训练模型。

BERT和GPT都不约而同使用了BooksCorpus的数据集，以及英文维基百科等。后续模型则扩充了使用的数据集。

从最初版本的GPT到GPT3，区别主要在于模型的大小和用于训练的数据量。

## 进化：从GPT到InstructGPT

经过以上的步骤之后，模型基本可以做到“熟读唐诗三百首，不会作诗也会吟”，然而其效果往往差强人意，离人工智能的目标还差的很远，甚至可能会产生对用户有害的输出。为此，OpenAI创造性地开创了InstructGPT。[这个链接](https://openai.com/index/instruction-following/) 解释了InstructGPT的工作流程，核心步骤如下：、

![InstructGPT](pic/instructgpt.png)

其核心步骤包括两个：

1. 监督微调（supervised fine-tuning）
2. 通过人类反馈进行强化学习（reinforcement learning from human feedback, RLHF）

从第一个GPT模型只有0.1b参数（b代表billion，十亿）开始，到2020年GPT-3的175b参数，再到2022年InstructGPT推出，从而进化到GPT-3.5（同样为175b参数），大模型的逐渐展现出了超乎想象的能力。到现在的GPT-4、GPT-4o，短时间内大模型的推力能力得到了极大的增强。与此同时，各种各样的大模型不断推出，包括Llama、GLM、Bard以及中文的通义千问、文心一言等大模型也有不错的效果。

## 警惕AI幻觉

虽然大语言模型已经表现出了强大的推理能力，然而值得注意的是，这里的推理仍然是在语言层面而不是在逻辑层面。所谓推理，无非是根据上下文不断的对下一个token进行预测，借助于足够大的参数量，大语言模型往往可以“背过”很多答案，但是要求大语言模型可以基于逻辑进行推理现在仍然难以实现。当然，从另一个层面说，人类的思考也是借助于语言工具，也许有一天完全基于语言的大模型可以进行逻辑推理，但是现在仍然难以达到。

一个比较简单而又广为人知的例子是问大模型：9.9和9.11哪个大？包括GPT在内的很多模型都回答错误。如果使用一些提问的技巧，GPT甚至会“振振有词”地为错误答案辩护。在一些计算题中同样有这个问题，比如如果问2 * 2等于多少，可以得到正确答案，但是如果问3666 * 5444等于多少，则可能会给出错误答案。

因而，不要完全信任大语言模型给出的结果。在实际使用过程中，应该借助更多的工具帮助大语言模型得到更好的答案，而不是完全依赖这个答案。

## 提示工程

由于大语言模型的不完美，往往我们的问题得不到最满意的回答，然而通过提高提示词的质量往往可以得到更好的回答。为此，提供正确的、高质量的提示词就可以让我们对AI工具的使用事半功倍，这被称为提示工程（prompt engineering）。常见的方法有：

* 提供详细、明确的上下文
  > 有一家公司为大飞机生产起落架，是不是新质生产力？
  > 新质生产力是创新起主导作用，摆脱传统经济增长方式、生产力发展路径，具有高科技、高效能、高质量特征，符合新发展理念的先进生产力质态。它由技术革命性突破、生产要素创新性配置、产业深度转型升级而催生。以劳动者、劳动资料、劳动对象及其优化组合的跃升为基本内涵，以全要素生产率大幅提升为核心标志，特点是创新，关键在质优，本质是先进生产力。根据以上定义，请问为大飞机生产起落架的公司符不符合新质生产力的要求？
* 明确任务
  > 请为比亚迪公司是否属于新质生产力打分，1-10分。
* 明确角色
  > 如果你是一个计量经济学老师，请写一个教学计划。
* 零样本思维链策略（zero-shot-CoT strategy）：让LLM逐步思考、给出过程和步骤
  > 3666 * 5444等于多少？请给出计算步骤
* 少样本学习（few-shot learning）甚至单样本学习（one-shot learning）
  > 1025=1，20480=2，360560101=3，5023512004=？
* 格式化输出
  > 请为比亚迪公司是否属于新质生产力打分，1-10分，必须为数字
* 重复指示：提示词中多次加入相同的指令，但是表述方式不相同，也可以通过负面来提示
  > 你作为投资人，需要考察比亚迪公司的新质生产力发展情况，需要进行一个打分。请你根据你所知道的情况，对比亚迪公司进行分析，针对不好的地方进行扣分，1-10分。不要给出理由，必须给出数字。
* 思维链、思维树提示：将复杂问题分解

# Ollama的部署与使用

## 部署

## 嵌入

## 对话

# 微调大模型

# Dify的部署和使用